In [1]:
from copy import deepcopy
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import datetime
import math
from tqdm import tqdm
from torchinfo import summary

from game import UTTT
from ai import *

In [2]:
prev_best = Player(n_sims=150) # path=r'C:\Users\dylan\Desktop\Code\Python\Ultimate-Tic-Tac-Toe\Models\20220403183827.pt')
current_player = prev_best.copy() # Player(n_sims=100)
board = UTTT()

In [13]:
board.move(2, 2)

In [14]:
board.print_board()

|     ||  X O||    X|
|     ||     ||     |
|     ||     ||     |
---------------------
|     ||     ||     |
|     ||     ||     |
|     ||     ||     |
---------------------
|     ||     ||     |
|     ||     ||     |
|     ||     ||     |
---------------------


In [15]:
board.board_features()

array([[[0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0

In [12]:
board.board_features()

array([[[0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1

In [3]:
summary(current_player.brain, input_size=(16, 3, 9, 9))

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNet                                --                        --
├─Conv2d: 1-1                            [16, 5, 7, 7]             140
├─MaxPool2d: 1-2                         [16, 5, 3, 3]             --
├─Conv2d: 1-3                            [16, 5, 3, 3]             30
├─MaxPool2d: 1-4                         [16, 5, 2, 2]             --
├─Linear: 1-5                            [16, 50]                  1,050
├─Linear: 1-6                            [16, 81]                  4,131
├─Linear: 1-7                            [16, 1]                   51
Total params: 5,402
Trainable params: 5,402
Non-trainable params: 0
Total mult-adds (M): 0.20
Input size (MB): 0.02
Forward/backward pass size (MB): 0.05
Params size (MB): 0.02
Estimated Total Size (MB): 0.09

In [ ]:
prev_best = Player(n_sims=150) # path=r'C:\Users\dylan\Desktop\Code\Python\Ultimate-Tic-Tac-Toe\Models\20220403183827.pt')
current_player = prev_best.copy() # Player(n_sims=100)
board = UTTT()

while True:
    data = gen_episodes(current_player, board, 25)
    current_player.train(data, batch_size=16, epochs=3)
    result = compete(prev_best, current_player, cutoff=0.6, matches=10)
    if result == current_player:
        print('New Winner!')
        current_player.save_brain(path=r'C:\Users\dylan\Desktop\Code\Python\Ultimate-Tic-Tac-Toe\Models\{}.pt'.format(datetime.datetime.now().strftime('%Y%m%d%H%M%S')))
        prev_best = current_player.copy()
